# Solving boundary-value ODEs

<a href="http://www.physics.ucla.edu/icnsp/Html/spong/spong.htm" target="_blank"><img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/stellarator.jpg" width=500px /></a>


## PHYS 2600: Scientific Computing

## Lecture 14

## Types of numerical ODEs

In math class, you learn a ton of vocabulary for _classifying_ ODEs: homogeneous, linear, separable...

For numerical solution, _we don't care_ about most of those math-class labels!  Instead, only one categorical division really matters for numerical ODEs:  __initial value problems__ (IVPs) versus __boundary value problems__ (BVPs). 

<img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/bvp-vs-ivp.png" width=600px />

In an IVP, we set all boundary conditions at a _single_ value of independent variable $x = x_0$ - the __initial value__.  In a BVP, boundary conditions are present at _two or more_ values of $x$.

We now know how to solve IVPs by _iteration_: start at $x_0$, discretize, find the solution by stepping out from $x_0$.

In general, __boundary-value problems are harder__ than IVPs, because they require "non-local" information about the solution.  If we try to step out from one boundary, we don't know if the solution will match at the other boundary.

Fortunately, hard doesn't mean impossible!  There are two common approaches to solving boundary-value ODEs: __shooting__, and __relaxation__.

## Shooting

Shooting exploits the fact that we _do_ know how to solve an ODE from an initial value, so we start by _partly_ matching the boundary conditions.

Consider a second-order ODE with $y(0) = 0$ and $y(x_1) = y_1$.  Whatever the final solution is, it can _also_ be uniquely described by the initial conditions $y(0) = 0, y'(0) = c$ for some unknown $c$.  So we can just _guess_ some $c$, "shoot" the solution from $y(0)$ to $y(x_1)$ by solving numerically, and see how close we get!

<img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/bvp-shooting.png" width=600px />

Now the problem is reduced to adjusting $c$ until we hit the other boundary.  In fact, for any given $c$ we have $y(x_1) = F(c)$, so this is just a root-finding problem for the (black box) function $F(c)$ - and we know how to solve those!


## Relaxation

Shooting has a couple of drawbacks; the most significant is _instability_.  If our solutions are highly sensitive to the initial guess, it may be very hard to make shooting converge.

A more robust techinque is __relaxation__, which operates _globally_ on the whole solution between our boundaries at 0 and $x_1$.  We again start with an initial guess, but our guess is for a complete $y(x)$ (ideally, one that matches the two boundaries, or at least comes close!)  

Our guess $y(x)$ will fail to satisfy the differential equation, but if we can propose a "relaxation step" that puts it _closer_ to solving the ODE, then we can iterate towards an actual solution.

<img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/bvp-relaxation.png" width=500px />


Implementing relaxation in practice is very tricky, and the algorithms are quite complex, so I'm not going to go into them.  (The ever-popular _Numerical Recipes_ has all of the details you could ever want.)

There is one relaxation-based BVP solver I know of: once again in SciPy, the function `scipy.integrate.solve_bvp`.  The documentation says that this function using a "collocation algorithm", which is a fancy way to do relaxation where we express the solution as a sum over a number of simple functions with unknown coefficients.

On the other hand, I don't know of any existing BVP solver that uses shooting in Python!  But that's alright, because shooting is just a simple combination of an IVP solver and a root-finder, and we have plenty of both to choose from.

<img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/clint-relax.jpg" width=300px style="float:right;" />

It might be tempting to use the built-in SciPy solver first, but it's definitely harder to use and understand than applying shooting.  Most professional numerical analysts go by the gunslinger's motto: "__shoot first, relax later.__" 

## Root finding

The basic problem is as follows: given the equation

$$
f(x) = 0,
$$

find all values $\{x_r\}$ that satisfy the equation.  (The solutions $\{x_r\}$ are the _roots_ or _zeroes_ of $f(x)$.)

Many problems in physics and math can be framed as root-finding!  Solving any equation $f(x) = g(x)$ is just root-finding $f(x) - g(x) = 0$.  Calculating $\sqrt{a}$ is equivalent to finding the roots of the equation

$$
y^2 - a = 0.
$$

There are many algorithms for root finding; here are three examples.

* __Brute force__ checks every number over a certain interval to see which ones satisfy the equation. 
* __Bisection__ uses a simple math argument to quickly narrow down a single root.
* __Newton-Raphson__ uses the derivative $f'(x)$ to extrapolate to where the nearest root is.  ("Heron's method" for the square root is actually Newton-Raphson!)

All of these methods are __iterative methods__: start with an initial guess $x_0$, and then improve the guess repeatedly ($x_0 \rightarrow x_1 \rightarrow x_2...$) until it's "close enough" to correct.  The __stopping condition__ defines when we are "close enough" in math terms.

There are Python modules that implement all of these and many more well-known algorithms for root finding.  Many are available in the [scipy.optimize](https://docs.scipy.org/doc/scipy/reference/optimize.html#root-finding) sub-module.

One rule of good programming is: _never re-invent the wheel!_  But this is a class, so our job is to see how these algorithms work on the inside, to save you from trouble when you try to use them.

The stopping condition is usually given in terms of some __error tolerance__ $\epsilon$.  This is, ideally, the error on our numerical solution: in other words, our answer $x_i$ and the true root $x_r$ should satisfy

$$
|x_i - x_r| < \epsilon.
$$

Unfortunately, we usually can't test this directly - we don't know $x_r$!  





Instead, we can check how close $f(x_i)$ is to zero using our bound:

$$
|f(x_i) - f(x_r)| = |f(x_i)| < \epsilon.
$$

Taylor expanding about $x_r$ lets us relate this to what we want:

$$
|f(x_i)| \approx |f'(x_r)| |x_i - x_r| < \epsilon
$$

which is _almost_ the same as $|x_i - x_r| < \epsilon$; we just have an extra constant, $|f'(x_r)|$.  As long as it isn't too big, bounding $|f(x_i)|$ is a good bound on the error for estimating $x_r$.

<img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/root-find-sketch.png" width=500px style="float:right;margin:20px" />

Here's a more visual way to see what is going on: we want the _horizontal_ distance to the root, but the best we can usually do is the _vertical_ distance.  They are related by the slope of $f(x)$ at $x_r$.

For most practical uses, we simply __set the error tolerance to be much smaller than any other sources of error__ in our problem, so we don't care if we're off by $\epsilon$ or $2\epsilon$.  (If making $\epsilon$ really small becomes too expensive, then you need to worry about this.)

## Tutorial 14

Let's solve the Schrodinger equation!  Connect to the server and load up `tut14`.

## Additional reading

If you're hungry for more advanced root-finding algorithms and an in-depth study of the problem in general, chapter 9 of the famous _Numerical Recipes_ has all the root-finding you can handle!  (The code examples, however, are not in Python.)